## Goal: understanding contributors

The term "community" in this context refers to the group of people contributing to Mozilla projects. Thus, this goal could be summarized as characterizing Mozilla community based on their contributors. A contributor will be understood as a person who performs an action that can be tracked in the set of considered data sources. For example: sending a commit, opening or closing a ticket. As they will be different depending on the data source, particular actions used in each analysis will be detailed within particular goals.

The main objective of this goal is to determine a set of characteristics of contributors:

  * Projects: to which projects they contribute.
  * Organizations: to which organizations they are affiliated
  * Gender: which one is their gender
  * Age: which one is their "age" in the project (time contributing)
  * Geographical origin: where do they come from

Those goals can be refined in the following questions:

**Questions**:

* Which projects can be identified?
* Which contributors have activity related to each project?
* Which organizations can be identified?
* Which contributors are affiliated to each organization?
* Which of those contributors are hired by Mozilla, and which are not?
* Which gender are contributors?
* How long have been contributors contributing?
* Where do contributors come from?

These questions can be answered with the following metrics/data:

**Metrics**:

* List of projects
* Contributors by project
* Number of contributors by project over time
* List of organizations
* Contributors by organization
* Number of contributors by organization over time
* Contributors by groups: hired by Mozilla, the rest
* Contributors by gender
* Number of contributors by gender over time
* Time of first and last commit for each contributor
* Length of period of activity for each contributor
* Contributors by time zone (when possible)
* Contributors by city name (when possible)

All the characeterizations of developers (by project, by organization, by hired by Mozilla/rest, by gender, by period of activity, by time zone, by city name) can be a discriminator / grouping factor for the metrics defined for the next goals. Most of these metrics can be made particular for each of the considered data sources.

### Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.



In [1]:
import pandas

import plotly as plotly
import plotly.graph_objs as go

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

In [2]:
def create_search(source):
    s = Search(using=es_conn, index=source)
    # TODO: Add bot and merges filtering.
    #s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
    s.params(timeout=30)
    return s

In [3]:
def print_result(result):
    """In case you need to check query response, call this function
    """
    print(result.to_dict()['aggregations'])

In [4]:
def print_df(result, group_field, value_field, group_column, value_column):
    df = pandas.DataFrame()

    df = df.from_dict(result.to_dict()['aggregations'][group_field]['buckets'])
    df = df.drop('doc_count', axis=1)
    df[value_field] = df[value_field].apply(lambda row: row['value'])
    df=df[['key', value_field]]
    df.columns = [group_column, value_column]

    return df

In [5]:
def stack_by(result, group_column, time_column, value_column, group_field, time_field, value_field):
    """Creates a dataframe based on group and time values
    """
    df = pandas.DataFrame(columns=[group_column, time_column, value_column])

    for b in result.to_dict()['aggregations'][group_field]['buckets']:
        for i in b[time_field]['buckets']:
            df.loc[len(df)] = [b['key'], i['key_as_string'], i[value_field]['value']]
    
    return df

In [6]:
def stack_by_cusum(result, group_column, time_column, value_column, group_field, time_field, value_field,\
                   staff_org_names, staff_org):
    authors_org_df = pandas.DataFrame(columns=[group_column, time_column, value_column])

    for b in result.to_dict()['aggregations'][group_field]['buckets']:
        key = b['key']
        if key in staff_org_names:
            key = staff_org
        else:    
            key  = 'Other'    

        print(b['key'], '->' ,key)

        for i in b[time_field]['buckets']:

            time = i['key_as_string']
            contributors = i[value_field]['value']

            if key in authors_org_df[group_column].unique() \
                and time in authors_org_df[authors_org_df[group_column] == key][time_column].tolist():

                authors_org_df.loc[(authors_org_df[group_column] == key) \
                                     & (authors_org_df[time_column] == time),\
                                   value_column] += contributors
                #print('1', key,  time, contributors)

            else:
                authors_org_df.loc[len(authors_org_df)] = [key, time, contributors]
                #print('2', key,  time, contributors)
    
    return authors_org_df


In [7]:
def print_stacked_bar(df, time_column, value_column, group_column):
    """Print stacked bar chart from dataframe based on time_field,
    grouped by group field.
    """
    plotly.offline.init_notebook_mode(connected=True)

    bars = []
    for group in df[group_column].unique():
        group_slice_df = df.loc[df[group_column] == group]
        bars.append(go.Bar(
            x=group_slice_df[time_column].tolist(),
            y=group_slice_df[value_column].tolist(),
            name=group))

    layout = go.Layout(
        barmode='stack'
    )

    fig = go.Figure(data=bars, layout=layout)
    plotly.offline.iplot(fig, filename='stacked-bar')

#### List of Projects

To get the list of projects we will query ES to retrieve the unique count of commits for each project. To do that, we bucketize data based on 'project' field (to a maximum of 100 projects, given by 'size' parameter set below).

In [8]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()

In [9]:
print_df(result=result, group_field='projects', value_field='commits', \
         group_column='Project', value_column='# Commits')

,Project,# Commits
0,mozilla,1905232
1,mozilla-services,148022
2,rust-lang,137277
3,servo,78712
4,mdn,12785
5,moztw,8625
6,mozilla-mobile,9344
7,aframevr,7263
8,mozilla-japan,5798
9,mozmar,5569


#### Contributors by Project

In [10]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('projects', 'terms', field='project', size=100)\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

In [11]:
print_df(result=result, group_field='projects', value_field='contributors', \
         group_column='Project', value_column='# Contributors')

,Project,# Contributors
0,mozilla,10991
1,mozilla-services,2128
2,rust-lang,2453
3,servo,1397
4,mdn,334
5,moztw,89
6,mozilla-mobile,109
7,aframevr,217
8,mozilla-japan,58
9,mozmar,73


#### Number of contributors by project over time

In [12]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
s.aggs.bucket('projects', 'terms', field='project', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()
            
df = stack_by(result=result, group_column='Project', time_column='Time', value_column='# Contributors',\
        group_field='projects', time_field='time', value_field='contributors')

In [13]:
print_stacked_bar(df=df, time_column='Time', value_column='# Contributors', group_column='Project')

#### List of organizations


In [14]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100)\
    .metric('commits', 'cardinality', field='hash', precision_threshold=100000)
result = s.execute()

In [15]:
print_df(result=result, group_field='organizations', value_field='commits', \
         group_column='Organization', value_column='# Commits')

,Organization,# Commits
0,Mozilla Staff,1374637
1,Community,970174
2,Mozilla Reps,631
3,Unknown,137


#### Contributors by organization

In [16]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('organizations', 'terms', field='author_org_name', size=100).\
    metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)
result = s.execute()

In [17]:
print_df(result=result, group_field='organizations', value_field='contributors', \
         group_column='Organization', value_column='# Contributors')

,Organization,# Contributors
0,Mozilla Staff,2084
1,Community,13083
2,Mozilla Reps,2
3,Unknown,77


#### Number of contributors by organization over time

In [18]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
s.aggs.bucket('org', 'terms', field='author_org_name', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()

authors_org_df = stack_by(result=result, group_column='Organization', time_column='Time', value_column='# Contributors',\
        group_field='org', time_field='time', value_field='contributors')

In [19]:
print_stacked_bar(df=authors_org_df, time_column='Time', value_column='# Contributors', group_column='Organization')

#### Contributors by groups: hired by Mozilla, the rest

In [20]:
s = create_search(source='git')

# Unique count of Commits by Project (max 100 projects)
s = s.filter('range', grimoire_creation_date={'gt': 'now/M-2y', 'lt': 'now/M'})
s.aggs.bucket('org', 'terms', field='author_org_name', size=10)\
    .bucket('time', 'date_histogram', field='grimoire_creation_date', interval='quarter')\
    .metric('contributors', 'cardinality', field='author_uuid', precision_threshold=100000)

result = s.execute()

authors_org_df = stack_by_cusum(result=result, group_column='Organization', time_column='Time',\
                                value_column='# Contributors', group_field='org', time_field='time',\
                                value_field='contributors', staff_org_names=['Mozilla Staff'],\
                                staff_org='Mozilla Staff')

Mozilla Staff -> Mozilla Staff
Community -> Other
Mozilla Reps -> Other
Unknown -> Other


In [21]:
print_stacked_bar(df=authors_org_df, time_column='Time', value_column='# Contributors', group_column='Organization')

#### Contributors by gender
**TODO**: Pending of running gender study over the data.

#### Number of contributors by gender over time
**TODO**: Pending of running gender study over the data.

#### Time of first and last commit for each contributor
**TODO** : provide plots similar to:
Attracted developers: https://analytics.mozilla.community:443/goto/1be7d078d3dda22bf2ed097d8e465fb9
Last commit developers: https://analytics.mozilla.community:443/goto/647ec69fc6b9d163827aa281ed3bee61

#### Length of period of activity for each contributor


#### Contributors by time zone (when possible)

https://analytics.mozilla.community:443/goto/9e930eefe59a7c90331d922887b2aee6

#### Contributors by city name (when possible)